### imports

In [206]:
import pjlsa_gsipro as pjlsa

import random
import time
import json
from datetime import datetime

import numpy as np
from scipy.interpolate import LinearNDInterpolator
import matplotlib.pyplot as plt

%matplotlib inline  

In [2]:
from pyda import SimpleClient
from pyda.data import DataFilter, TimingSelector
from pyda_rda3 import RdaProvider

In [3]:
import jpype
with pjlsa.LSAClientGSI().java_api():
    from cern.lsa.client import ServiceLocator, ContextService, ParameterService, TrimService, SettingService
    from cern.accsoft.commons.value import ValueFactory
    from cern.lsa.domain.settings import TrimRequest, ContextSettingsRequest, Settings, SettingPartEnum
    from cern.lsa.domain.settings.type import  BeamProcessPurposes
    from de.gsi.lsa.domain.settings import GsiBeamProcessPurpose
    from java.util import Set, Collections

 INFO [07 Nov 2023 13:44:19,769] (PropertyConfig.java) - Processing property file http://asl156.acc.gsi.de:58080/application/profile/pro/lsa.properties.
 INFO [07 Nov 2023 13:44:19,848] (PropertyConfig.java) - Populating system properties.
 INFO [07 Nov 2023 13:44:19,849] (PropertyConfig.java) - Setting property lsa.masp.deviceName=ZMA_ZS
 INFO [07 Nov 2023 13:44:19,849] (PropertyConfig.java) - Setting property contextCategories=MD,OPERATIONAL,TEST
 INFO [07 Nov 2023 13:44:19,850] (PropertyConfig.java) - Setting property lsa.bss.dataSupplyEnabled=true
 INFO [07 Nov 2023 13:44:19,850] (PropertyConfig.java) - Setting property lsa.trim.concurrency.enabled=true
 INFO [07 Nov 2023 13:44:19,850] (PropertyConfig.java) - Setting property cavity.phase.changeatmerging=true
 INFO [07 Nov 2023 13:44:19,851] (PropertyConfig.java) - Setting property lsa.sis18.noBeamStrict=true
 INFO [07 Nov 2023 13:44:19,851] (PropertyConfig.java) - Setting property cavity.h2.maxlimit=8000
 INFO [07 Nov 2023 13:44:1

In [4]:
ts = ServiceLocator.getService(TrimService)
cs = ServiceLocator.getService(ContextService)
ps = ServiceLocator.getService(ParameterService)
ss = ServiceLocator.getService(SettingService)

lsa.server.properties = lsa-gsi-pro-server.properties
DEBUG [07 Nov 2023 13:44:20,562] (AbstractApplicationContext.java) - Refreshing org.springframework.context.annotation.AnnotationConfigApplicationContext@710b18a6
DEBUG [07 Nov 2023 13:44:20,593] (DefaultSingletonBeanRegistry.java) - Creating shared instance of singleton bean 'org.springframework.context.annotation.internalConfigurationAnnotationProcessor'
DEBUG [07 Nov 2023 13:44:20,675] (PropertySourcesPropertyResolver.java) - Found key 'lsa.server.properties' in PropertySource 'systemProperties' with value of type String
DEBUG [07 Nov 2023 13:44:20,686] (PropertySourcesPropertyResolver.java) - Found key 'lsa.server.properties' in PropertySource 'systemProperties' with value of type String
DEBUG [07 Nov 2023 13:44:20,703] (PropertySourcesPropertyResolver.java) - Found key 'lsa.server.properties' in PropertySource 'systemProperties' with value of type String
DEBUG [07 Nov 2023 13:44:20,869] (DefaultSingletonBeanRegistry.java) - Cre

In [5]:
TERMINAL_COLORS = [
"\u001b[31m", "\u001b[32m", "\u001b[33m", "\u001b[34m", "\u001b[35m", "\u001b[36m", "\u001b[37m", "\u001b[31;1m", "\u001b[32;1m", "\u001b[33;1m", "\u001b[34;1m", "\u001b[35;1m",
]

TERMINAL_COLOR_RESET = "\u001b[0m"


## fetch pattern and related beam processes

In [69]:
pattern = cs.findPattern("SIS18_SLOW_...")

DEBUG [07 Nov 2023 14:57:38,913] (RbaTokenLookup.java) - Rba token not found.
DEBUG [07 Nov 2023 14:57:38,913] (Slf4jLogger.java) - [ApplicationFeignService#checkServerVersionCompatibility] ---> GET http://lsapro00a.acc.gsi.de:52330/application/checkServerVersionCompatibility/18.2.1 HTTP/1.1
DEBUG [07 Nov 2023 14:57:38,913] (Slf4jLogger.java) - [ApplicationFeignService#checkServerVersionCompatibility] Accept: application/json
DEBUG [07 Nov 2023 14:57:38,913] (Slf4jLogger.java) - [ApplicationFeignService#checkServerVersionCompatibility] Accept-Charset: utf-8
DEBUG [07 Nov 2023 14:57:38,913] (Slf4jLogger.java) - [ApplicationFeignService#checkServerVersionCompatibility] Content-Type: application/json;charset=UTF-8
DEBUG [07 Nov 2023 14:57:38,914] (Slf4jLogger.java) - [ApplicationFeignService#checkServerVersionCompatibility] x-client-id: app=/home/bphy/ccaliari/lnx/miniforge3/envs/pjlsa_pro/bin/python3.9;uid=ccaliari;host=asl753.acc.gsi.de;pid=740027;
DEBUG [07 Nov 2023 14:57:38,914] (Slf4

In [70]:
beamProcesses = list(pattern.getBeamProcesses())

for i, p in enumerate(beamProcesses):
    print(i, p.getName())

0 SIS18_SLOW_HADES_20231106_171354.C1.GHHTMU1_TO_GTH3MU1.BEAMOUT_INIT.1
1 SIS18_SLOW_HADES_20231106_171354.C1.GTE3MU1_TO_GHHTMU1.BEAMOUT_INIT.1
2 SIS18_SLOW_HADES_20231106_171354.C1.GTH3MU1_TO_GTP1MU1.BEAMOUT_INIT.1
3 SIS18_SLOW_HADES_20231106_171354.C1.GTP1MU1_TO_HADES.BEAMOUT_INIT.1
4 SIS18_SLOW_HADES_20231106_171354.C1.GTS1MU1_TO_GTE3MU1.BEAMOUT_INIT.1
5 SIS18_SLOW_HADES_20231106_171354.C1.SIS18_TO_GTS1MU1.BEAMOUT_INIT.1
6 SIS18_SLOW_HADES_20231106_171354.C1.GTK7BC1L_TO_PLTKMH2__GS.BEAMOUT_INIT.1
7 SIS18_SLOW_HADES_20231106_171354.C1.PLTKMH2_TO_SIS18__GS.BEAMOUT_INIT.1
8 SIS18_SLOW_HADES_20231106_171354.C1.SIS18_RING.BEAMOUT_INIT.1
9 SIS18_SLOW_HADES_20231106_171354.C1.GTK7BC1L_TO_PLTKMH2__GS.TRANSFER_INJECTION.1
10 SIS18_SLOW_HADES_20231106_171354.C1.PLTKMH2_TO_SIS18__GS.TRANSFER_INJECTION.1
11 SIS18_SLOW_HADES_20231106_171354.C1.SIS18_RING.RING_INJECTION.1
12 SIS18_SLOW_HADES_20231106_171354.C1.GTK7BC1L_TO_PLTKMH2__GS.BEAMOUT_RESET.1
13 SIS18_SLOW_HADES_20231106_171354.C1.PLTKMH2_

In [78]:
FAIR_SELECTOR = beamProcess_to_trim.getUser()
FAIR_SELECTOR

'FAIR.SELECTOR.C=14:T=300:S=8:P=61'